# Checking Datasets

**Author: Shahzad Sanjrani**

**Date: 21.11.24**

Notebook to just sanity check the input of the h5 input/output.

In [43]:
import h5py
import numpy as np
import os
import mplhep as hep
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt

In [44]:
store_dir = "/nfs/dust/cms/user/sanjrani/SPANet_Investigations/investigation2/pepper_analysis/output/h4t_systematics/spanet"

# infile = "input/genstudies_2017_jpt20_GENRECO_training/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/TTTT_TuneCP5_13TeV-amcatnlo-pythia8_even_train.h5"
infile = "input/genstudies_2017_jpt20_GENRECO_training/TTZprimeToTT_M-500_Width4_TuneCP5_13TeV-madgraph-pythia8/TTZprimeToTT_M-500_Width4_TuneCP5_13TeV-madgraph-pythia8_even_train.h5"

### What do we want to check?

Some of the jet distributions.

Some of the assignment percentages.

Some of the masses.

In [45]:
def plot_hepfig(figsize=(10,10)):
    ''' plt.figure but use the mplhep package to make it look nice '''
    hep.style.use("CMS")
    mpl.rcParams["font.size"] = 20
    plt.figure(figsize=figsize)

In [46]:
def get_order_of_mag(min_value, counter_max = 10):
    order_of_mag = 1
    counter = 0
    while order_of_mag > min_value:
        order_of_mag = order_of_mag / 10
        counter += 1
        if counter > counter_max:
            break
    return order_of_mag

In [47]:
def get_ymin(counts, density=False, yscale='linear'):
    ymin = 0
    if yscale == 'log' or density == True:
        ymin = 1
        if np.min(counts) < ymin:
            ymin = get_order_of_mag(np.min(counts))

In [48]:
def plot_input(input_key, input_attribute, file, bins=None, yscale='linear'):

    try:
        inarray_mask = ak.Array(file[f"INPUTS/{input_key}/MASK"])
    except:
        inarray_mask = None

    inarray = ak.Array(file[f"INPUTS/{input_key}/{input_attribute}"])
    if inarray_mask is not None:
        inarray = inarray[inarray_mask]

    plot_hepfig()
    if bins is None:
        # default is printing 10 bins between min and max
        bins = np.linspace(np.min(inarray), np.max(inarray), 10)
        
    counts, _, _ = plt.hist(ak.flatten(inarray), bins=bins, label=f"{input_key}:{input_attribute}")
    plt.yscale(yscale)
    plt.xlim((bins[0],bins[-1]))
    plt.ylim((0, np.max(counts)*1.35))
    plt.xlabel(f"N. {input_key}")
    plt.ylabel(f"{input_attribute}")

    return None
    

In [49]:
def resonance_checks(resonance_reco, resonance_keys, label=None, save=None, check_name=None, fpath=None):
    '''
    Plot fraction of events of how many events where N res in res_keys are reconstructed
    '''
    res_reco = np.array( [resonance_reco[res] for res in resonance_keys] )
    res_n = np.sum(res_reco.astype(int), axis=0)
    plot_hepfig()

    yscale, density, bins = 'linear', True, np.arange(0, len(resonance_keys)+2)
    counts, _, _ = plt.hist( res_n, bins=bins, density=density, label=label)
    ymin = get_ymin(counts, density=density, yscale=yscale)
    plt.ylim((ymin, np.max(counts)*1.35))
    plt.xlim((bins[0], bins[-1]))
    plt.xlabel(f"N. reconstructable")
    plt.ylabel(f"Fraction of events")
    plt.yscale(yscale)
    plt.legend()

    if save is not None and check_name is not None and fpath is not None:
        # save is dir
        # check_name is suffix
        # fpath is h5 file path
        fname = os.path.basename(fpath)
        fname = fname.replace(".h5", "")
        savepath = os.path.join(save, f"{fname}_{check_name}.pdf")
        plt.savefig(savepath)
        plt.close()

def plot_targets(file, tttt_checks=True, yscale='linear', save=None, fpath=None):
    '''
    assumes we have the following: (cba to generalise)
    - {'ti':['q1','q2','b'], ..} for i in [1,2,3,4]
    - {'zpti':['q1','q2','b'], 'nzpti':['q1','q2','b'],..} for i in [1,2]
    '''
    resonance_keys = {}
    for k in list(file["TARGETS"].keys()):
        resonance_keys[k] = list(file[f"TARGETS/{k}"].keys())

    # generalised
    resonance_reco = {}
    for resonance in resonance_keys:
        daughters = np.array([ file[f"TARGETS/{resonance}/{daughter}"] for daughter in resonance_keys[resonance] ])
        reco = np.all( daughters >= 0, axis = 0)
        resonance_reco[resonance] = reco

    plot_hepfig()
    x_pos, x_labels = np.arange(0,len(list(resonance_reco.keys())))+0.5, list(resonance_reco.keys())
    heights = [ np.sum(resonance_reco[k]) for k in resonance_keys ]
    
    plt.bar(x_pos, heights, width=0.5, tick_label=x_labels)
    plt.ylim((0, np.max(heights)*1.35))
    plt.xlabel(f"Top")
    plt.ylabel(f"N. reconstructable")

    check_name = "resonance_reco"
    if save is not None and fpath is not None:
        fname = os.path.basename(fpath)
        fname = fname.replace(".h5", "") 
        savepath = os.path.join(save, f"{fname}_{check_name}.pdf")
        plt.savefig(savepath)
        plt.close()

    if tttt_checks:

        # check how many hnt
        tops = ['t1','t2','t3','t4']
        resonance_checks(resonance_reco, resonance_keys=tops, label='SM tops', save=save, check_name="SM_tops", fpath=fpath)

        # check how many hnt (zp)
        tops = ['zpt1','zpt2','nzpt1','nzpt2']
        resonance_checks(resonance_reco, resonance_keys=tops, label='Zp, non-Zp tops', save=save, check_name="Zp_nZp_tops", fpath=fpath)

        # check how many zp tt
        tops = ['zpt1','zpt2']
        resonance_checks(resonance_reco, resonance_keys=tops, label='Zp tops', save=save, check_name="Zp_tops", fpath=fpath)

        # check how many non z' tt
        tops = ['nzpt1','nzpt2']
        resonance_checks(resonance_reco, resonance_keys=tops, label='non-Zp tops', save=save, check_name="nZp_tops", fpath=fpath)

In [66]:
def combine_group(group_four_vectors):
    '''
    Given dictionary in format : {'pt':[..], 'phi':[..], 'eta':[..], 'mass':[..], 'match':[..]}
    - calculate invariant mass
    '''
    ## -- Initial settings -- ##
    # 1. convert to np arrays
    for v in group_four_vectors: 
        group_four_vectors[v] = np.array(group_four_vectors[v])

    # 2. check if calculation possible
    rep_cartesian = ['px','py','pz','energy']
    rep_polar = ['pt','eta','phi','mass']
    if not(set(rep_cartesian) < set(list(group_four_vectors.keys()))) and not (set(rep_polar) < set(list(group_four_vectors.keys()))):
        raise ValueError(f"Can't calculate M: both {rep_cartesian} and {rep_polar} not given")

    # 3. check if we need to convert polar -> cartesian
    convert = not (set(rep_cartesian) < set(list(group_four_vectors.keys())))
    
    ## -- Convert to px, py, pz, E --- ##
    if convert:
        group_four_vectors['px'] = group_four_vectors['pt'] * np.cos(group_four_vectors['phi'])
        group_four_vectors['py'] = group_four_vectors['pt'] * np.sin(group_four_vectors['phi'])
        group_four_vectors['pz'] = group_four_vectors['pt'] * np.sinh(group_four_vectors['eta'])
        group_four_vectors['energy'] = np.sqrt( group_four_vectors['px']**2 + group_four_vectors['py']**2 + group_four_vectors['pz']**2 + group_four_vectors['mass']**2 )
    
    ## -- Sum the four vectors and get mass -- ##
    px = np.sum(group_four_vectors['px'], axis=0)
    py = np.sum(group_four_vectors['py'], axis=0)
    pz = np.sum(group_four_vectors['pz'], axis=0)
    energy = np.sum(group_four_vectors['energy'], axis=0)
    mass = np.sqrt( energy**2 - (px**2 + py**2 + pz**2) )
    
    if 'match' in group_four_vectors: 
        match = np.all( group_four_vectors['match'], axis=0 )
    else:
        match = np.full_like(px, True)

    return {"px":px, "py":py, "pz":pz, "energy":energy, "mass":mass, "match":match}

def plot_mass(mass, match, label, save=None, fpath=None):
    '''
    Given mass array, match mask array --> plot it
    '''
    plot_hepfig()
    bins = np.arange(0, 5000, 50) 
    counts, _, _ = plt.hist(mass[match], bins=bins, label=label)
    plt.xlabel(f"Mass [GeV]")
    plt.ylabel(f"N. particles")
    plt.xlim((bins[0],bins[-1]))
    plt.ylim((0,1.35*np.max(counts)))
    plt.legend()

    if save is not None and fpath is not None:
        fname = os.path.basename(fpath)
        fname = fname.replace(".h5", "") 
        savepath = os.path.join(save, f"{fname}_{label}.pdf")
        plt.savefig(savepath)
        plt.close()

def resonance_invariant_mass(file, input_2_target, targets_set=None, save=None, fpath=None):
    '''
    calculate invariant mass of resonance particles:
    - input_2_target = dict in format {'res_i':'input_group_a', ..}
    - targets_set = (optional) dict in format {'big_res_x':['res_i','res_j'], 'big_res_y':[...]}
    '''
    
    ## -- Initial settings -- ##
    rep_cartesian = ['px','py','pz','energy']
    rep_polar = ['pt','eta','phi','mass']
    
    resonance_four_vectors_all = {}

    for resonance, resonance_input in input_2_target.items():
        input_components = list(file[f"INPUTS/{resonance_input}"].keys())

        # -- check if possible -- #
        if set(rep_cartesian) < set(input_components):
            four_vectors = rep_cartesian
        elif set(rep_polar) < set(input_components):
            four_vectors = rep_polar
        else:
            print(f"Cannot calculate {resonance} with input {resonance_input}")
            continue
            
        # -- get four vectors from resonance -- #
        daughters = list(file[f"TARGETS/{resonance}"].keys())
        daughters_four_vectors = { component: [] for component in four_vectors }
        
        for component in daughters_four_vectors:
            for daughter in daughters:
                
                input_component = np.array(file[f"INPUTS/{resonance_input}/{component}"])
                daughter_labels = np.array(file[f"TARGETS/{resonance}/{daughter}"])
                
                input_component_d = input_component[ np.arange(input_component.shape[0]), daughter_labels ] # e.g., get jet.pt's for that daughter
                daughters_four_vectors[component].append(input_component_d)

        daughters_four_vectors['match'] = [ np.array(file[f"TARGETS/{resonance}/{daughter}"]) > -1  for daughter in daughters ]

        # get 4vec (cartesian) dict from daughters for resonance
        resonance_four_vectors = combine_group(daughters_four_vectors)

        ## -- add to collection of resonances -- ##
        resonance_four_vectors_all[resonance] = resonance_four_vectors

        ## -- plot -- ##
        plot_mass(resonance_four_vectors['mass'], resonance_four_vectors['match'], resonance, save=save, fpath=fpath)

    ## -- Repeat above for groups of resonances -- ##
    if targets_set is not None:
        
        four_vectors = rep_cartesian # assumed because of combine_group output
        
        for bigres, bigres_group in targets_set.items():
            
            bigres_group_4vecs = { component:[] for component in four_vectors }
            for component in bigres_group_4vecs:
                for res in bigres_group:
                    bigres_group_4vecs[component].append( resonance_four_vectors_all[res][component] )
            bigres_group_4vecs['match'] = [ resonance_four_vectors_all[res]['match'] for res in bigres_group ]

            bigres_4vecs = combine_group(bigres_group_4vecs)

            plot_mass(bigres_4vecs['mass'], bigres_4vecs['match'], bigres, save=save, fpath=fpath)
    

## Make your checks here

In [67]:
infile_path = os.path.join(store_dir, infile) # full path
save_dir = os.path.join(os.path.dirname(infile_path), "plots") # where to save plots
os.makedirs(save_dir, exist_ok=True)

with h5py.File(infile_path, "r") as file:

    # plot_input("Jets", "pt", file)
    # plot_targets(file, tttt_checks=True, save=save_dir, fpath=infile_path)

    # some manual work involved
    input_2_target = { res:'Jets' for res in ['t1','t2','t3','t4','zpt1','zpt2','nzpt1','nzpt2'] }
    targets_set = {'Zp':['zpt1','zpt2'], 'nZp':['nzpt1','nzpt2']}
    resonance_invariant_mass(file, input_2_target, targets_set=targets_set, save=save_dir, fpath=infile_path)